<a href="https://colab.research.google.com/github/BugsRugs/AlgoExpertSolutions/blob/master/nine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

In [ ]:
#3 card suited nine 5x
#3 card nine 3x
#natural nine suited 3x
#natural nine 2x
#3 card suited 3x
#2 card suited 2x
#2 card pair 2x
#value 8 -> 0 beats
#variables for genomes: personal(suited?, pair?, val is = 9, = 8, =7...) dealer(none)
#first we create intial genes, after we have the population, create the fitness score for each gemone,
#then rebreed the new population with parents and do process again


def fitness(genome, numHands):
  score = 0
  for i in range(numHands):
    playerHand = []
    dealerHand = []
    dealCards(playerHand, dealerHand)
    features = setFeatures(playerHand)
    y = 0
    for idx, x in enumerate(features):
      y += x * genome[idx]
    if y > 0:
      playerHand.append(hit(playerHand))
    dealerHit(dealerHand)
    score += scoreHands(playerHand, dealerHand)
  return score
  

def scoreHands(playerHand, dealerHand):
  values = []
  pval = sum(x[0] for x in playerHand) % 10
  dval = sum(x[0] for x in dealerHand) % 10
  multiplier = 0
  handToCheck = []
  if pval > dval:
    multiplier = 1
    handToCheck = [playerHand]
  elif pval < dval:
    multiplier = -1
    handToCheck = [dealerHand]
  else:
    handToCheck = [playerHand , dealerHand]
  for hand in handToCheck:
    c1, c2, c3, s1, s2, s3 = 0, 0, 0, '', '', ''
    c1, s1 = hand[0]
    c2, s2 = hand[1]
    if len(hand) != 2:
      c3, s3 = hand[2]
    handval = (c1 + c2 + c3) % 10
    if s1 == s2 and s2 == s3 and handval == 9:
      values.append(1)
    elif s3 != '' and handval == 9:
      values.append(2)
    elif handval == 9 and s1 == s2 and s3 == '':
      values.append(3)
    elif handval == 9:
      values.append(4)
    elif s1 == s2 and s2 == s3:
      values.append(5) 
    elif s1 == s2 and s3 == '':
      values.append(6)
    elif c1 == c2 and c3 == '':
      values.append(7)
    else:
      values.append(8)
  points = {1:5, 2:3, 3:3, 4:2, 5:3, 6:2, 7:2, 8:1}
  if len(values) == 1:
    return points[values.pop()]*multiplier
  elif len(values) == 2:
      if values[0] < values[1]:
        return points[values[0]]
      elif values[0] > values[1]:
        return points[values[1]]*-1
      else:
        return 0
  else:
    raise Exception("something went wrong scoring cards")

    


def dealerHit(dealerHand):
  weights = [-2.438, 1.298, -5.267, -1.746, -3.02, -4.064, -1.473, 2.607, 7.506, 8.245, 6.534, 0.931]
  features = setFeatures(dealerHand)
  y = 0
  for idx, x in enumerate(features):
    y += x * weights[idx]
  if y > 0:
    dealerHand.append(hit(dealerHand))


def hit(playerHand):
  seen = {}
  seen[playerHand[0]] = 0
  seen[playerHand[1]] = 0
  while True:
    card = deck[random.randint(0, 51)]
    if card in seen:
      continue
    else:
      return card



def setFeatures(playerHand):
  #suited, pair, nine, 8, 7, 6, 5, 4, 3, 2, 1, 0
  features = [0]*12
  c1, s1 = playerHand[0]
  c2, s2 = playerHand[1]
  handVal = (c1 + c2) % 10
  if s1 == s2: features[0] = 1
  if c1 == c2: features[1] = 1
  if handVal == 9: features[2] = 1
  elif handVal == 8: features[3] = 1
  elif handVal == 7: features[4] = 1
  elif handVal == 6: features[5] = 1
  elif handVal == 5: features[6] = 1
  elif handVal == 4: features[7] = 1
  elif handVal == 3: features[8] = 1
  elif handVal == 2: features[9] = 1
  elif handVal == 1: features[10] = 1
  elif handVal == 0: features[11] = 1
  else: raise Exception("Card val isn't a number?")
  return features


def dealCards(playerHand, dealerHand):
  seen = {}
  while len(playerHand) + len(dealerHand) < 4:
      if len(playerHand) < 2:
        card = deck[random.randint(0, 51)]
        if card in seen:
          continue
        else:
          playerHand.append(card)
          seen[card] = 0
      if len(dealerHand) < 2:
        card = deck[random.randint(1, 51)]
        if card in seen:
          continue
        else:
          dealerHand.append(card)
          seen[card] = 0


def generateDeck():
  deck = []
  for i in range(1, 14):
    if i >= 10:
      i = 0
    deck.append((i, 's'))
    deck.append((i, 'h'))
    deck.append((i, 'c'))
    deck.append((i, 'd'))
  return deck


def initializeGenes(populationSize, numGenes):
  r = []
  for j in range(populationSize):
    o = []
    for i in range(numGenes):
      o.append(round(random.uniform(-10.00, 10.00), 3))
    r.append(o)
  return r

def reproduce(genome1, genome2, populationSize):
  # population = []
  # for i in range(populationSize):
  #   breakpoint = random.randint(1, 11)
  #   newGene = []
  #   for i in range(breakpoint):
  #     newGene.append(round(((genome1[i] + genome2[i]) / 2.000 ), 3))
  #   for j in range(breakpoint, 12):
  #     newGene.append(round(((genome1[j] + genome2[j]) / 2.000 ), 3))
  #   if random.randint(1, 10) == 4:
  #     newGene = []
  #     for gene in genome1:
  #       if random.randint(1, 3) == 1:
  #         newGene.append(round(random.uniform(-10.00, 10.00), 3))
  #       else:
  #         newGene.append(gene)
  #   population.append(newGene)
  # return population
  population = []
  for i in range(populationSize):
    breakpoint = random.randint(1, 11)
    newGene = []
    for x in range(breakpoint):
      if random.randint(1, 20) == 2:
        newGene.append(round(random.uniform(-10.00, 10.00), 3)) 
        continue 
      newGene.append(genome1[x])
    for x in range(breakpoint, 12):
      newGene.append(genome2[x])
    population.append(newGene)
  return population

def runSimulations():
  population = initializeGenes(100, 12)
  maxScore = 0
  weights = []
  for i in range(100):
    populationScores = []
    top1, top2, maximum = 0, 0, float('-inf')
    for person in population:
      populationScores.append(fitness(person, 100))
    for idx, score in enumerate(populationScores):
        if score >= maximum:
          top2 = top1
          top1 = idx
          maximum = score
          weights.append(population[top1])
    maxScore = max(maximum, maxScore)
    population = reproduce(population[top1], population[top2], 100)
  print(maxScore)
  print(weights.pop())
  #suited, pair, nine, 8, 7, 6, 5, 4, 3, 2, 1, 0



deck = generateDeck()
runSimulations()

52
[-6.966, 8.764, -3.945, -2.879, 6.345, 3.591, 4.981, 3.804, 6.4, 6.222, -4.172, -4.318]
